In [22]:
# auto relode
%load_ext autoreload
%autoreload 2

# Ctrl model dataset

In [ ]:
import pickle

In [54]:
# dataset

class CtrlModelDataset:
    def __init__(self, path_to_pickle_files, num_past_time_steps, num_future_time_steps):
        self.path_to_pickle_files = path_to_pickle_files
        self.num_past_time_steps = num_past_time_steps
        self.num_future_time_steps = num_future_time_steps

        pickl_files = glob.glob(self.path_to_pickle_files + '/*')
        
        # read all pickle files
        self.data = []
        for pickle_file in pickl_files:
            with open(pickle_file, 'rb') as f:
                data = pickle.load(f)
                self.data.extend(data)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        past_inputs = self.data[idx]['past_inputs'][0, :, :]
        past_outputs = self.data[idx]['past_outputs'][0, :, :]
        past_interals = self.data[idx]['past_interals'][0, :, :]
        future_outputs = self.data[idx]['future_infered_outputs'][0, :, :self.num_future_time_steps]
        next_input = self.data[idx]['future_optimized_inputs'][0, :, 0]
    
        return past_inputs, past_outputs, past_interals, future_outputs, next_input

In [56]:
ctrl_model_dataset = CtrlModelDataset('./ctrl_datasets', 30, 30)

In [57]:
len(ctrl_model_dataset)

501

In [64]:
# dataloader
from torch.utils.data import DataLoader

ctrl_model_dataloader = DataLoader(ctrl_model_dataset, batch_size=2, shuffle=True)

# Ctrl model

In [58]:
from ctrl_model import CtrlModel
import torch

In [71]:
ctrl_model = CtrlModel(
    num_past_time_steps=30,
    num_future_time_steps=30,
    num_input_vars=2,
    num_output_vars=1,
    num_internal_vars=2
)

In [65]:
# get a sample from the dataset
past_inputs, past_outputs, past_interals, future_outputs, next_input = ctrl_model_dataset[0]

In [66]:
ctrl_model(
    past_inputs=past_inputs.unsqueeze(0),
    past_outputs=past_outputs.unsqueeze(0),
    past_interals=past_interals.unsqueeze(0),
    future_outputs=future_outputs.unsqueeze(0)
)


tensor([[[-0.0720],
         [ 0.2677]]], grad_fn=<UnsqueezeBackward0>)

In [73]:
# train using pytorch lightning

import pytorch_lightning as pl

trainer = pl.Trainer(
    gpus=1,
    max_epochs=10
)

trainer.fit(ctrl_model, ctrl_model_dataloader)

/home/nils/miniconda3/envs/i2dl/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:446: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type   | Params
-----------------------------------
0 | conv1   | Conv1d | 100   
1 | conv2   | Conv1d | 620   
2 | conv3   | Conv1d | 5.3 K 
3 | fc1     | Linear | 9.2 K 
4 | conv_f1 | Conv1d | 12    
5 | conv_f2 | Conv1d | 30    
6 | fc_new2 | Linear | 9.1 K 
7 | fc_new3 | Linear | 620   
8 | fc_new4 | Linear | 42    
-----------------------------------
25.0 K    Trainable params
0         Non-trainable params
25.0 K    Total params
0.100     Total estimated model p

Training: 0it [00:00, ?it/s]

/home/nils/repos/w22-test-bench/controllers/nn_1/dev/ctrl_model/ctrl_model.py:198: UserWarning: Using a target size (torch.Size([2, 2])) that is different to the input size (torch.Size([2, 2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = torch.nn.functional.mse_loss(next_predicted_input, next_input)
/home/nils/repos/w22-test-bench/controllers/nn_1/dev/ctrl_model/ctrl_model.py:198: UserWarning: Using a target size (torch.Size([1, 2])) that is different to the input size (torch.Size([1, 2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = torch.nn.functional.mse_loss(next_predicted_input, next_input)
`Trainer.fit` stopped: `max_epochs=10` reached.


In [74]:
# check the average loss
trainer.test(ctrl_model, ctrl_model_dataloader)
# [{'test_loss': 5.459200859069824}

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss            4.981675148010254
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 4.981675148010254}]